# Tecnicas para limpieza de datos

1. Eliminar duplicados
`drop_duplicates()`
2. Eliminar caracteres no deseados
`str.replace()`
3. Corrección del tipo de dato
df[ “campo” ].astype(float) [texto del enlace](https://)
4. Manejo de datos faltantes
`dropna()`
`fillna()`
5. Normalización de datos
`.str.strip().lower()`
6. Filtrado de datos
`df [ df[“campo”] + condición ]`


## Importamos algunos datasets para explorar

In [2]:
# Importar la librería Pandas
import pandas as pd

In [ ]:
# @title
# Montar la unidad
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title
# Verificar que los archivos csv se encuentren en la carpeta datasets
import os
os.listdir("/content/drive/MyDrive/datasets")

In [ ]:
# @title
# Importamos el Dataset satis (customer satisfaction)
df_satis = pd.read_csv("/content/drive/MyDrive/datasets/satis_clientes.csv")
df_satis.info()

In [66]:
import pandas as pd

# Importamos el Dataset Netflix
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shivamb/netflix-shows")

print("\nPath to folder of dataset files => :", path)
df_netflix = pd.read_csv(path + '/netflix_titles.csv') # Added the filename here

#nf_url = "https://github.com/alex-degarate/TT-2C2025-Data-Analitycs-Notebooks/blob/main/datasets/netflix_titles.csv"
#df_nf2 = pd.read_csv(nf_url)

print("\n")



Using Colab cache for faster access to the 'netflix-shows' dataset.

Path to folder of dataset files => : /kaggle/input/netflix-shows




In [72]:

#df_netflix.info()
print("\n")
#print(df_netflix.iloc[:, 0:3])
#df_netflix[['title', 'country', 'release_year']].head()

titulo = df_netflix['title']

print(len(titulo))

# obtengo un substring de ese dato para cada valor del indice
df_netflix['title'] = df_netflix['title'].str.slice(0, 25)
df_netflix['country'] = df_netflix['country'].str.slice(0, 20)

display( df_netflix[['show_id', 'title', 'country']].head(100))

# print("\n Duplicados => ",df_netflix.duplicated().sum())




8807


,show_id,title,country
0,s1,Dick Johnson Is Dead,United States
1,s2,Blood & Water,South Africa
2,s3,Ganglands,NaN
3,s4,Jailbirds New Orleans,NaN
4,s5,Kota Factory,India
...,...,...,...
95,s96,The Circle,"United States, Unite"
96,s97,If I Leave Here Tomorrow:,United States
97,s98,Kid Cosmic,United States
98,s99,Octonauts: Above & Beyond,United Kingdom


In [26]:
# Save the DataFrame to a CSV file
df_netflix.to_csv("netflix_titles.csv", index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importamos el Dataset de pacientes con sus mediciones de temperatura
df_pacientes = pd.read_csv('https://docs.google.com/spreadsheets/d/1-rUn4TUwpGrLE1DH8moeiR5eSyeF-jOOTpfhRgZxVIQ/gviz/tq?tqx=out:csv&sheet=')
df_pacientes.info()


## Identificación de datos duplicados

[duplicated()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html)

### Dataframe satis

In [ ]:
df_satis.info()

In [ ]:
# Veamos si hay duplicados
df_satis.duplicated().sum()

In [ ]:
# Veamos cuantos registros duplicados hay
df_satis.duplicated(subset=["Empresa", "Calificación"]).sum()

In [ ]:
# Contar valores unicos
df_satis.value_counts(subset=["Empresa", "Calificación"]).sort_values(ascending=False)

In [ ]:
# Visualizar los duplicados con diferentes argumentos
df_satis[df_satis.duplicated(keep=False)].sort_values(by='Empresa')

In [ ]:
# Aplicar filtros
df_satis[df_satis['Empresa'] == "Keebler Inc"]

In [ ]:
df_satis[df_satis.duplicated(subset=["Empresa", "Fecha"],keep=False)].sort_values(by='id')

In [ ]:
df_satis[df_satis.duplicated(subset=df_satis.columns[1:],keep=False)].sort_values(by='Empresa')

### Dataframe Pacientes

In [ ]:
# Veamos cuantos registros duplicados hay
df_pacientes.duplicated().sum()

In [ ]:
# Si hay, entonces los listamos (keep, first, last, False)
df_pacientes[df_pacientes.duplicated(subset=["nombre"], keep=False)].sort_values(by='nombre')

### Dataframe Netflix

In [ ]:
df_netflix.duplicated().sum()

In [ ]:
df_netflix[df_netflix.duplicated()]

## Tratamiento de datos duplicados
[Pandas drop_duplicates()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)

### Dataset Satis

In [ ]:
df_satis.info()

In [ ]:
df_satis_pp1 = df_satis.drop_duplicates(subset=["Empresa", "Calificación"], keep="first")
df_satis_pp1.info()

In [ ]:
df_netflix.duplicated().sum()

In [ ]:
df_satis_pp1[df_satis_pp1.duplicated(subset=["Empresa", "Calificación"], keep=False)].sort_values(by='Empresa')


## Exploración de datos nulos

In [ ]:
df_satis.info()

In [ ]:
# Ver si hay valores nulos, en general o en columnas especificas
#df_satis.isnull().sum()
#df_satis[["Comentarios", "Calificación"]].isnull().sum()
df_satis.isnull().any(axis=1).sum()

In [ ]:
# Visualizar los registros con celdas null / NaN
df_satis[df_satis.isnull().any(axis=1)][["Comentarios", "Calificación"]]

## Tratamiento de datos nulos

### drop

`dropna` se usa para eliminar registros que contengan datos nulos
<BR>
[Pandas dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)

* how: para especificar si (any / all) celdas deben tener NaN
* thresh: indicar la cantidad de NaN para ejecutar el drop (no se puede combinar con any)
* subset: indicar las columnas a evaluar
* inplace: si guarda los cambios en el dataframe o retorna una copia

Analizar y comparar que sucede cuando aplicamos dropna a todas las columnas o a algunas especificas.

In [ ]:
# Eliminar un registro si alguna las columnas contienen NaA
df_satis_pp2 = df_satis.dropna()
df_satis_pp2.info()

In [ ]:
# Eliminar un registro si alguna o tdas las columnas contienen NaA
df_satis_pp2 = df_satis.dropna(how="all")
df_satis_pp2.info()

In [ ]:
# Eliminar un registro solo evaluando NaN en las columnas indicadas
df_satis_pp2 = df_satis.dropna(subset=["Comentarios", "Calificación"])
df_satis_pp2.info()

In [ ]:
# Eliminar una columna si contiene NaN
df_satis_pp2 = df_satis.dropna(axis=1) # no accepta subset
df_satis_pp2.info()

### fill

`fillna` se utiliza para completar el dato faltante
<BR>
[Pandas fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html)
<BR>
* df.fillna(0)  se completa con un valor fijo
* df.fillna(method="ffill")  completa con el valor del registro anterior
* df.fillna(method="bfill") completa con el valor del registro previo
* df["col"].fillna(df["col"].mean()) completa con la media de la columna
* df.fillna({"col1": 0, "col2": "desconocido"})

1. completamos con un valor fijo

In [ ]:
# usamos el atributo value con un valor fijo en 0
df_satis_pp3 = df_satis.fillna(value = 0)
df_satis_pp3.info()

In [ ]:
# También podríamos usar un diccionario
df_satis_pp3 = df_satis.fillna({"Calificación": 0, "Comentarios": "Sin dato"})

In [ ]:
# listamos los primeros registros
df_satis_pp3.head(10)

In [ ]:
# Eventualmente ajustamos el tipo de dato
df_satis_pp3 = df_satis_pp3.astype({"Comentarios": "string"})
df_satis_pp3.info()

In [ ]:
# Podemos aplicar filtros
df_satis_pp3[df_satis_pp3["Comentarios"]=="0"]

2. Completamos con el valor próximo

In [ ]:
df_satis_pp3 = df_satis.fillna(method = "ffill")
df_satis_pp3.info()

3. Completamos con el valor previo

In [ ]:
df_satis_pp3 = df_satis.fillna(method = "bfill")
df_satis_pp3.info()

4. Completamos con la media, meadiana, moda

In [ ]:
df_satis_pp3 = df_satis.fillna(df_satis.mean(numeric_only=True))
df_satis_pp3.info()

In [ ]:
# calcular la media de la columna Calificación
media_calif = df_satis["Calificación"].mode()[0] # mean() / median() / mode()[0]

# aplicar fillna con diccionario
df_satis_pp3 = df_satis.fillna({
    "Calificación": media_calif,
    "Comentarios": "Sin dato"
})

In [ ]:
df_satis_pp3.head(5)

Analicemos el dataset pacientes

In [ ]:
df_pacientes.info()

In [ ]:
# Completamos con la media de la columna
df_pacientes_pp1 = df_pacientes.fillna(df_pacientes.mean(numeric_only=True))
df_pacientes_pp1.info()

In [ ]:
# Completamos con la media de la fila usando lambda

# seleccionar solo las columnas d1...d10
cols = df_pacientes.columns[1:]   # todas excepto 'nombre'

# aplicar moda por fila
df_pacientes_pp1[cols] = df_pacientes[cols].apply(
    lambda row: row.fillna(row.mean()), axis=1
)

In [ ]:
# Completamos con la media de la fila usando T (traspuesta)
df_pacientes_pp1[cols] = df_pacientes[cols].T.fillna(df_pacientes[cols].mean(axis=1)).T

In [ ]:
#df_pacientes.head()
df_pacientes_pp1.head()

Veamos el concepto de Traspuesta

In [ ]:
df = pd.DataFrame({
    "A": [1, 2, 3],
    "B": [4, 5, 6]
})
print("Original:")
print(df)

print("\nTranspuesta:")
print(df.T)

## Normalización de datos

## Ajustar tipo de dato

### astype()

* df["col"] = df["col"].astype(int)
* df["col"] = df["col"].astype(float)
* df["col"] = df["col"].astype(str)

In [ ]:
df_satis_pp4 = df_satis.copy()
df_satis_pp4["Comentarios"] = df_satis["Comentarios"].astype("string")

In [ ]:
df_satis_pp4.info()

### to_numeric()

In [ ]:
pd.to_numeric(df["col"], errors="coerce")

### to_datetime()

In [ ]:
type(df_satis["Fecha"][0])
df_satis["Fecha"][0]

In [ ]:
# pd.to_datetime(df_satis.Fecha)
pd.to_datetime(df_satis["Fecha"], format="%d/%m/%Y")


## Filtrado de datos

In [ ]:
df_satis.info()

In [ ]:
# Con una sola columna
df_satis["Empresa"]

In [ ]:
# Con varias columnas
df_satis[["Empresa", "Fecha"]]

In [ ]:
df_satis[df_satis["Empresa"] == "Kuhn-Fay"]

In [ ]:
# Con condición
df_satis[[("Empresa") =="Alpha" & ("Comentarios" > "3")]]